In [ ]:
# Setup and imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import re
import json
from typing import Dict, List, Tuple, Any
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")


In [ ]:
@dataclass
class SystemComponent:
    """Represents a component in a three-tier system."""
    name: str
    tier: str  # 'presentation', 'business', 'data'
    technology: str
    code: str
    config: str
    patterns: List[str]
    dependencies: List[str]

@dataclass
class ThreeTierSystem:
    """Represents a complete three-tier system."""
    name: str
    description: str
    components: List[SystemComponent]
    architecture_decisions: List[str]
    tech_stack: Dict[str, str]

print("Data structures defined!")


In [ ]:
def generate_mock_systems() -> List[ThreeTierSystem]:
    """Generate mock three-tier system implementations with different approaches."""
    
    # System 1: Modern React + Node.js + PostgreSQL
    system1_components = [
        SystemComponent(
            name="React Frontend",
            tier="presentation",
            technology="React",
            code="""
import React, { useState, useEffect } from 'react';
import axios from 'axios';

const UserDashboard = () => {
    const [users, setUsers] = useState([]);
    const [loading, setLoading] = useState(true);
    
    useEffect(() => {
        const fetchUsers = async () => {
            try {
                const response = await axios.get('/api/users');
                setUsers(response.data);
            } catch (error) {
                console.error('Error fetching users:', error);
            } finally {
                setLoading(false);
            }
        };
        fetchUsers();
    }, []);
    
    return (
        <div className="dashboard">
            <h1>User Dashboard</h1>
            {loading ? <div>Loading...</div> : 
                <div className="user-grid">
                    {users.map(user => (
                        <UserCard key={user.id} user={user} />
                    ))}
                </div>
            }
        </div>
    );
};
            """,
            config="""{
    "scripts": {
        "start": "react-scripts start",
        "build": "react-scripts build"
    },
    "dependencies": {
        "react": "^18.0.0",
        "axios": "^0.27.0",
        "styled-components": "^5.3.0"
    }
}""",
            patterns=["Component-Based Architecture", "Hooks Pattern", "Async/Await", "Error Boundaries"],
            dependencies=["axios", "react-router"]
        ),
        SystemComponent(
            name="Express API",
            tier="business",
            technology="Node.js/Express",
            code="""
const express = require('express');
const { Pool } = require('pg');
const cors = require('cors');

const app = express();
const pool = new Pool({
    connectionString: process.env.DATABASE_URL
});

app.use(cors());
app.use(express.json());

// User routes
app.get('/api/users', async (req, res) => {
    try {
        const result = await pool.query('SELECT * FROM users ORDER BY created_at DESC');
        res.json(result.rows);
    } catch (err) {
        console.error(err);
        res.status(500).json({ error: 'Internal server error' });
    }
});

app.post('/api/users', async (req, res) => {
    const { name, email } = req.body;
    try {
        const result = await pool.query(
            'INSERT INTO users (name, email) VALUES ($1, $2) RETURNING *',
            [name, email]
        );
        res.status(201).json(result.rows[0]);
    } catch (err) {
        console.error(err);
        res.status(500).json({ error: 'Failed to create user' });
    }
});

module.exports = app;
            """,
            config="""{
    "dependencies": {
        "express": "^4.18.0",
        "pg": "^8.7.0",
        "cors": "^2.8.5",
        "helmet": "^5.1.0"
    }
}""",
            patterns=["RESTful API", "MVC Pattern", "Connection Pooling", "Error Handling Middleware"],
            dependencies=["pg", "express", "cors"]
        ),
        SystemComponent(
            name="PostgreSQL Database",
            tier="data",
            technology="PostgreSQL",
            code="""
-- Users table schema
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    email VARCHAR(255) UNIQUE NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Indexes for performance
CREATE INDEX idx_users_email ON users(email);
CREATE INDEX idx_users_created_at ON users(created_at);

-- Trigger for updated_at
CREATE OR REPLACE FUNCTION update_updated_at_column()
RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = CURRENT_TIMESTAMP;
    RETURN NEW;
END;
$$ language 'plpgsql';

CREATE TRIGGER update_users_updated_at 
    BEFORE UPDATE ON users 
    FOR EACH ROW 
    EXECUTE FUNCTION update_updated_at_column();
            """,
            config="""{
    "database": "userapp",
    "host": "localhost",
    "port": 5432,
    "ssl": false,
    "pool": {
        "min": 2,
        "max": 10
    }
}""",
            patterns=["Relational Database", "ACID Compliance", "Indexing Strategy", "Triggers"],
            dependencies=[]
        )
    ]
    
    system1 = ThreeTierSystem(
        name="Modern Web App",
        description="React frontend with Node.js API and PostgreSQL database",
        components=system1_components,
        architecture_decisions=[
            "Single Page Application (SPA) architecture",
            "RESTful API design",
            "Relational database with ACID properties",
            "Stateless backend services",
            "Component-based frontend architecture"
        ],
        tech_stack={
            "frontend": "React",
            "backend": "Node.js/Express",
            "database": "PostgreSQL",
            "deployment": "Docker"
        }
    )
    
    return [system1]

# Generate initial system
mock_systems = generate_mock_systems()
print(f"Generated {len(mock_systems)} mock systems")
print(f"System 1: {mock_systems[0].name}")
print(f"Components: {[c.name for c in mock_systems[0].components]}")


In [ ]:
# Add a second system with different architecture for comparison
def add_microservices_system():
    """Add a microservices-based system for comparison."""
    
    # System 2: Vue.js + Python Flask + MongoDB (Microservices)
    system2_components = [
        SystemComponent(
            name="Vue.js Frontend",
            tier="presentation",
            technology="Vue.js",
            code="""
<template>
  <div class="user-management">
    <h1>User Management</h1>
    <div v-if="loading" class="loader">Loading...</div>
    <div v-else>
      <user-form @user-created="fetchUsers" />
      <user-table :users="users" @user-updated="fetchUsers" />
    </div>
  </div>
</template>

<script>
import { ref, onMounted } from 'vue'
import userService from '@/services/userService'

export default {
  name: 'UserManagement',
  setup() {
    const users = ref([])
    const loading = ref(true)
    
    const fetchUsers = async () => {
      try {
        loading.value = true
        const response = await userService.getUsers()
        users.value = response.data
      } catch (error) {
        console.error('Failed to fetch users:', error)
      } finally {
        loading.value = false
      }
    }
    
    onMounted(() => {
      fetchUsers()
    })
    
    return {
      users,
      loading,
      fetchUsers
    }
  }
}
</script>
            """,
            config="""{
  "dependencies": {
    "vue": "^3.2.0",
    "vue-router": "^4.0.0",
    "vuex": "^4.0.0",
    "axios": "^0.27.0"
  },
  "devDependencies": {
    "@vue/cli-service": "^5.0.0"
  }
}""",
            patterns=["Composition API", "Single File Components", "Reactive Data", "Service Layer"],
            dependencies=["vue-router", "vuex", "axios"]
        ),
        SystemComponent(
            name="Flask User Service",
            tier="business",
            technology="Python/Flask",
            code="""
from flask import Flask, request, jsonify
from flask_pymongo import PyMongo
from bson.objectid import ObjectId
from datetime import datetime
import os

app = Flask(__name__)
app.config["MONGO_URI"] = os.getenv("MONGO_URI", "mongodb://localhost:27017/userdb")
mongo = PyMongo(app)

@app.route('/api/users', methods=['GET'])
def get_users():
    try:
        users = list(mongo.db.users.find().sort("created_at", -1))
        for user in users:
            user['_id'] = str(user['_id'])
        return jsonify(users), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/users', methods=['POST'])
def create_user():
    try:
        data = request.get_json()
        user = {
            "name": data.get("name"),
            "email": data.get("email"),
            "created_at": datetime.utcnow(),
            "updated_at": datetime.utcnow()
        }
        result = mongo.db.users.insert_one(user)
        user['_id'] = str(result.inserted_id)
        return jsonify(user), 201
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/users/<user_id>', methods=['PUT'])
def update_user(user_id):
    try:
        data = request.get_json()
        update_data = {
            "name": data.get("name"),
            "email": data.get("email"),
            "updated_at": datetime.utcnow()
        }
        mongo.db.users.update_one(
            {"_id": ObjectId(user_id)},
            {"$set": update_data}
        )
        return jsonify({"message": "User updated successfully"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True, port=5001)
            """,
            config="""{
  "dependencies": {
    "Flask": "2.0.1",
    "Flask-PyMongo": "2.3.0",
    "python-dotenv": "0.19.0"
  },
  "environment": {
    "FLASK_ENV": "development",
    "MONGO_URI": "mongodb://localhost:27017/userdb"
  }
}""",
            patterns=["Microservices", "Document-based Storage", "RESTful API", "Dependency Injection"],
            dependencies=["flask", "pymongo", "python-dotenv"]
        ),
        SystemComponent(
            name="MongoDB Database",
            tier="data",
            technology="MongoDB",
            code="""
// User collection schema (flexible document structure)
{
  "_id": ObjectId,
  "name": String,
  "email": String,
  "created_at": Date,
  "updated_at": Date,
  "profile": {
    "avatar": String,
    "bio": String,
    "preferences": {
      "notifications": Boolean,
      "theme": String
    }
  },
  "metadata": {
    "ip_address": String,
    "user_agent": String,
    "last_login": Date
  }
}

// Indexes for performance
db.users.createIndex({ "email": 1 }, { unique: true })
db.users.createIndex({ "created_at": -1 })
db.users.createIndex({ "name": "text", "profile.bio": "text" })

// Aggregation pipeline for user analytics
db.users.aggregate([
  {
    $group: {
      _id: {
        $dateToString: { format: "%Y-%m-%d", date: "$created_at" }
      },
      count: { $sum: 1 }
    }
  },
  { $sort: { "_id": 1 } }
])
            """,
            config="""{
  "database": "userdb",
  "host": "localhost",
  "port": 27017,
  "options": {
    "useNewUrlParser": true,
    "useUnifiedTopology": true
  },
  "indexes": [
    { "fields": { "email": 1 }, "options": { "unique": true } },
    { "fields": { "created_at": -1 } }
  ]
}""",
            patterns=["Document Database", "Flexible Schema", "Aggregation Pipeline", "Text Search"],
            dependencies=[]
        )
    ]
    
    system2 = ThreeTierSystem(
        name="Microservices Web App",
        description="Vue.js frontend with Python Flask microservices and MongoDB",
        components=system2_components,
        architecture_decisions=[
            "Microservices architecture",
            "Document-based data storage",
            "Service-oriented design",
            "Flexible schema approach",
            "Composition API frontend pattern"
        ],
        tech_stack={
            "frontend": "Vue.js",
            "backend": "Python/Flask",
            "database": "MongoDB",
            "deployment": "Kubernetes"
        }
    )
    
    return system2

# Add the second system
system2 = add_microservices_system()
mock_systems.append(system2)

print(f"Total systems: {len(mock_systems)}")
for i, system in enumerate(mock_systems):
    print(f"System {i+1}: {system.name} - {system.description}")
